In [1]:
# !pip install opencv-python
# !pip install numpy
# !pip install pandas

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import os

## openpose 모델로드 및 gpu 설정

In [3]:
device = "gpu" # please change it to "gpu" if the model needs to be run on cuda.

protoFile = "pose_deploy_linevec.prototxt"
weightsFile = "pose_iter_440000.caffemodel"
nPoints = 18
# COCO Output Format
keypointsMapping = ['Nose', 'Neck', 'R-Sho', 'R-Elb', 'R-Wr', 'L-Sho', 
                    'L-Elb', 'L-Wr', 'R-Hip', 'R-Knee', 'R-Ank', 'L-Hip', 
                    'L-Knee', 'L-Ank', 'R-Eye', 'L-Eye', 'R-Ear', 'L-Ear']

POSE_PAIRS = [[1,2], [1,5], [2,3], [3,4], [5,6], [6,7],
              [1,8], [8,9], [9,10], [1,11], [11,12], [12,13],
              [1,0], [0,14], [14,16], [0,15], [15,17],
              [2,17], [5,16] ]

# index of pafs correspoding to the POSE_PAIRS
# e.g for POSE_PAIR(1,2), the PAFs are located at indices (31,32) of output, Similarly, (1,5) -> (39,40) and so on.
mapIdx = [[31,32], [39,40], [33,34], [35,36], [41,42], [43,44], 
          [19,20], [21,22], [23,24], [25,26], [27,28], [29,30], 
          [47,48], [49,50], [53,54], [51,52], [55,56], 
          [37,38], [45,46]]

colors = [ [0,100,255], [0,100,255], [0,255,255], [0,100,255], [0,255,255], [0,100,255],
         [0,255,0], [255,200,100], [255,0,255], [0,255,0], [255,200,100], [255,0,255],
         [0,0,255], [255,0,0], [200,200,0], [255,0,0], [200,200,0], [0,0,0]]

net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

if device == "cpu":
    net.setPreferableBackend(cv2.dnn.DNN_TARGET_CPU)
    print("Using CPU device")
elif device == "gpu":
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
    print("Using GPU device")

Using GPU device


## openpose 함수

In [4]:
# Find the Keypoints using Non Maximum Suppression on the Confidence Map
def getKeypoints(probMap, threshold=0.1):
    
    mapSmooth = cv2.GaussianBlur(probMap,(3,3),0,0)

    mapMask = np.uint8(mapSmooth>threshold)
    keypoints = []
    
    #find the blobs
    contours, _ = cv2.findContours(mapMask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    #for each blob find the maxima
    for cnt in contours:
        blobMask = np.zeros(mapMask.shape)
        blobMask = cv2.fillConvexPoly(blobMask, cnt, 1)
        maskedProbMap = mapSmooth * blobMask
        _, maxVal, _, maxLoc = cv2.minMaxLoc(maskedProbMap)
        keypoints.append(maxLoc + (probMap[maxLoc[1], maxLoc[0]],))

    return keypoints

# Find valid connections between the different joints of a all persons present
def getValidPairs(output, frameWidth, frameHeight, detected_keypoints):
    valid_pairs = []
    invalid_pairs = []
    n_interp_samples = 10
    paf_score_th = 0.1
    conf_th = 0.7
    # loop for every POSE_PAIR
    for k in range(len(mapIdx)):
        # A->B constitute a limb
        pafA = output[0, mapIdx[k][0], :, :]
        pafB = output[0, mapIdx[k][1], :, :]
        pafA = cv2.resize(pafA, (frameWidth, frameHeight))
        pafB = cv2.resize(pafB, (frameWidth, frameHeight))

        # Find the keypoints for the first and second limb
        candA = detected_keypoints[POSE_PAIRS[k][0]]
        candB = detected_keypoints[POSE_PAIRS[k][1]]
        nA = len(candA)
        nB = len(candB)

        # If keypoints for the joint-pair is detected
        # check every joint in candA with every joint in candB 
        # Calculate the distance vector between the two joints
        # Find the PAF values at a set of interpolated points between the joints
        # Use the above formula to compute a score to mark the connection valid
        
        if( nA != 0 and nB != 0):
            valid_pair = np.zeros((0,3))
            for i in range(nA):
                max_j=-1
                maxScore = -1
                found = 0
                for j in range(nB):
                    # Find d_ij
                    d_ij = np.subtract(candB[j][:2], candA[i][:2])
                    norm = np.linalg.norm(d_ij)
                    if norm:
                        d_ij = d_ij / norm
                    else:
                        continue
                    # Find p(u)
                    interp_coord = list(zip(np.linspace(candA[i][0], candB[j][0], num=n_interp_samples),
                                            np.linspace(candA[i][1], candB[j][1], num=n_interp_samples)))
                    # Find L(p(u))
                    paf_interp = []
                    for k in range(len(interp_coord)):
                        paf_interp.append([pafA[int(round(interp_coord[k][1])), int(round(interp_coord[k][0]))],
                                           pafB[int(round(interp_coord[k][1])), int(round(interp_coord[k][0]))] ]) 
                    # Find E
                    paf_scores = np.dot(paf_interp, d_ij)
                    avg_paf_score = sum(paf_scores)/len(paf_scores)
                    
                    # Check if the connection is valid
                    # If the fraction of interpolated vectors aligned with PAF is higher then threshold -> Valid Pair  
                    if ( len(np.where(paf_scores > paf_score_th)[0]) / n_interp_samples ) > conf_th :
                        if avg_paf_score > maxScore:
                            max_j = j
                            maxScore = avg_paf_score
                            found = 1
                # Append the connection to the list
                if found:            
                    valid_pair = np.append(valid_pair, [[candA[i][3], candB[max_j][3], maxScore]], axis=0)

            # Append the detected connections to the global list
            valid_pairs.append(valid_pair)
        else: # If no keypoints are detected
#             print("No Connection : k = {}".format(k))
            invalid_pairs.append(k)
            valid_pairs.append([])
#     print(valid_pairs)
    return valid_pairs, invalid_pairs

# This function creates a list of keypoints belonging to each person
# For each detected valid pair, it assigns the joint(s) to a person
# It finds the person and index at which the joint should be added. This can be done since we have an id for each joint
def getPersonwiseKeypoints(valid_pairs, invalid_pairs, keypoints_list):
    # the last number in each row is the overall score 
    personwiseKeypoints = -1 * np.ones((0, 19))

    for k in range(len(mapIdx)):
        if k not in invalid_pairs:
            partAs = valid_pairs[k][:,0]
            partBs = valid_pairs[k][:,1]
            indexA, indexB = np.array(POSE_PAIRS[k])

            for i in range(len(valid_pairs[k])): 
                found = 0
                person_idx = -1
                for j in range(len(personwiseKeypoints)):
                    if personwiseKeypoints[j][indexA] == partAs[i]:
                        person_idx = j
                        found = 1
                        break

                if found:
                    personwiseKeypoints[person_idx][indexB] = partBs[i]
                    personwiseKeypoints[person_idx][-1] += keypoints_list[partBs[i].astype(int), 2] + valid_pairs[k][i][2]

                # if find no partA in the subset, create a new subset
                elif not found and k < 17:
                    row = -1 * np.ones(19)
                    row[indexA] = partAs[i]
                    row[indexB] = partBs[i]
                    # add the keypoint_scores for the two keypoints and the paf_score 
                    row[-1] = sum(keypoints_list[valid_pairs[k][i,:2].astype(int), 2]) + valid_pairs[k][i][2]
                    personwiseKeypoints = np.vstack([personwiseKeypoints, row])
    return personwiseKeypoints

## 폴더 생성

In [5]:
#name 폴더 만들기
def make_dir(root_dir, dir_name):
    folder_list = []
    for i in dir_name:
        try: 
            if i not in folder_list:
                print("{0}/{1}".format(root_dir, i))
                os.makedirs("{0}/{1}".format(root_dir, i))
                folder_list.append(dir_name)

        except:
            pass
    return folder_list

## 파일 검색

In [6]:
import pathlib
import os
import ntpath

def find_file_fullname(fileDir):
    file_name = []
    for name in os.listdir(fileDir):
        file_name.append(name)
    return file_name

## 파트 분할

In [7]:
def take_not_nan(key1, key2, idx, part_data, part_nan, part_seq):
    if not np.isnan(key1) and not np.isnan(key2):
        d = np.array([key1, key2, idx], dtype=np.float32)
        part_data.append(d)
        return 0
    else:
        part_nan += 1
        if part_nan > 10: #오차 허용 프레임
            if len(part_data) > 30:
                append_seq_data(part_data, part_seq)
                return 100
            else:
                return 15
        return part_nan
        
def append_seq_data(part_data, part_seq):
    seq_length = 30
    part_data = np.array(part_data)
    
    for seq in range(len(part_data) - seq_length):
        part_seq.append(part_data[seq:seq + seq_length])
    part_data = []

## 영상 처리

In [13]:
#cap = cv2.VideoCapture(0)

def video_preprocessing(video_path, save_path, filename):
    cap = cv2.VideoCapture(video_path)
    data = []
    position = []
    last_position = []
    final_position_index = -1
    
    ret, image1 = cap.read()
    while cap.isOpened():
        t0 = cv2.getTickCount()
        if ret == False:
            break
            
        frameWidth = image1.shape[1]
        frameHeight = image1.shape[0]

        # Fix the input Height and get the width according to the Aspect Ratio
        inHeight = 368
        inWidth = int((inHeight/frameHeight)*frameWidth)
        inpBlob = cv2.dnn.blobFromImage(image1, 1.0 / 255, (inWidth, inHeight), (0, 0, 0), swapRB=False, crop=False)

        net.setInput(inpBlob)
        output = net.forward()

        i = 0
        probMap = output[0, i, :, :]
        probMap = cv2.resize(probMap, (frameWidth, frameHeight))

        detected_keypoints = []
        keypoints_list = np.zeros((0,3))
        keypoint_id = 0
        threshold = 0.1

        for part in range(nPoints):
            probMap = output[0,part,:,:]
            probMap = cv2.resize(probMap, (image1.shape[1], image1.shape[0]))
            keypoints = getKeypoints(probMap, threshold)
            keypoints_with_id = []
            for i in range(len(keypoints)):
                keypoints_with_id.append(keypoints[i] + (keypoint_id,))
                keypoints_list = np.vstack([keypoints_list, keypoints[i]])
                keypoint_id += 1

            detected_keypoints.append(keypoints_with_id)

        valid_pairs, invalid_pairs = getValidPairs(output, frameWidth, frameHeight, detected_keypoints)
        personwiseKeypoints = getPersonwiseKeypoints(valid_pairs, invalid_pairs, keypoints_list)
        
        
        #사람 구분
        detected_Obj = []
        last_position += position
        position = []
        keylist = np.empty((len(personwiseKeypoints), 13, 2))

        for i in range(1, 14):
            for n in range(len(personwiseKeypoints)):
                index = personwiseKeypoints[n][np.array(POSE_PAIRS[i])]
                if -1 in index:
                    continue
                A = np.int32(keypoints_list[personwiseKeypoints[n][i].astype(int)])
                keylist[n][i - 1] = A[:-1] #keylist는 목부터 발까지 좌표 값만 저장됨

        for n in range(len(personwiseKeypoints)):
            short_distance = 10000
            if int(keylist[n][0][0]) == 0:
                continue
            else:
                #여기서 벡터값 계산
                joint = np.empty((13, 2))
                for j in range(13):
                    try:
                        joint[j] = [keylist[n][j][0], keylist[n][j][1]]
                    except:
                        pass

                # Compute angles between joints
                v1 = joint[[0, 1, 2, 0, 4, 5, 0, 7, 8, 0, 10, 11], :2] # Parent joint
                v2 = joint[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], :2] # Child joint
                v = v2 - v1 # [12, 2]
                # Normalize v
                v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                # Get angle using arcos of dot product
                angle = np.arccos(np.einsum('nt,nt->n',
                    v[[0, 1, 3, 4, 6, 7, 9, 10],:], 
                    v[[1, 2, 4, 5, 7, 8, 10, 11],:]))

                angle = np.degrees(angle) # Convert radian to degree

                angle_label = np.array([angle], dtype=np.float32)

                count = -1
                last_p_index = -1
                short_p_index = -1
                for last_p in range(len(last_position)):
                    temp_x = keylist[n][0][0] - last_position[last_p][0]
                    temp_y = keylist[n][0][1] - last_position[last_p][1]
                    temp_distance = temp_x * temp_x + temp_y * temp_y
                    if temp_distance < short_distance:
                        short_distance = temp_distance
                        last_p_index = last_p
                        position_index = last_position[last_p][2]
                if last_p_index == -1:
                    final_position_index+=1
                    position.append([keylist[n][0][0], keylist[n][0][1], final_position_index, 0])

                    angle_label = np.append(angle_label, final_position_index)
                    detected_Obj.append(angle_label)
                    #여기서 벡터 배열에 삽입
                else:
                    position.append([keylist[n][0][0], keylist[n][0][1], position_index, 0])
                    del last_position[last_p_index]

                    angle_label = np.append(angle_label, position_index)
                    detected_Obj.append(angle_label)
                    #여기서 벡터 배열에 삽입

        data.append(detected_Obj)            

        del_list = []         
        for gone in range(len(last_position)):
            last_position[gone][3] += 1
            if last_position[gone][3] > 1:
                del_list.append(last_position[gone])
#         print(del_list)
        if len(del_list) > 0:
            for del_position in del_list:
                last_position.remove(del_position)
        
        t1 = cv2.getTickCount()
        ret, image1 = cap.read()
        print ("took %f seconds." % ((t1-t0)/cv2.getTickFrequency()))
        
    data = np.array(data)
    np.save(os.path.join(save_path + 'p_video', f'raw_{filename}_'), data)
    
    cap.release()
    cv2.destroyAllWindows()

# 실행

In [14]:
now = '/p_video/'
root_dir = 'C:/Users/BVer/are_you_ok'
video_dir = root_dir + '/datavideo' + now

try: #name 폴더 만들기
    os.makedirs("./preprocessing" + now)
except:
    pass

save_path = root_dir + "/preprocessing/"

print('preprocessing start')
video_name = []
video_name = find_file_fullname(video_dir)
for i in video_name:
    video_path = video_dir + i
    print(i)
    video_preprocessing(video_path, save_path, i)
    print(video_path)

print('----- finish -----')

preprocessing start
kicking_k1.mp4
took 0.945914 seconds.
took 0.940451 seconds.
took 0.925401 seconds.
took 0.899472 seconds.
took 0.887595 seconds.
took 0.923927 seconds.
took 0.915163 seconds.
took 0.924900 seconds.
took 0.940657 seconds.
took 0.941468 seconds.
took 0.927375 seconds.
took 0.905199 seconds.
took 0.925745 seconds.
took 0.922372 seconds.
took 0.891427 seconds.
took 0.907708 seconds.
took 0.905748 seconds.
took 0.906652 seconds.
took 0.939266 seconds.
took 0.913194 seconds.


c:\users\bver\miniconda3\envs\ayo\lib\site-packages\numpy\linalg\linalg.py:2506: RuntimeWarning: overflow encountered in multiply
  s = (x.conj() * x).real
c:\users\bver\miniconda3\envs\ayo\lib\site-packages\ipykernel_launcher.py:84: RuntimeWarning: divide by zero encountered in true_divide


took 0.943156 seconds.


c:\users\bver\miniconda3\envs\ayo\lib\site-packages\ipykernel_launcher.py:101: RuntimeWarning: overflow encountered in double_scalars


took 0.947364 seconds.
took 0.924489 seconds.
took 0.901105 seconds.


c:\users\bver\miniconda3\envs\ayo\lib\site-packages\ipykernel_launcher.py:84: RuntimeWarning: invalid value encountered in true_divide


took 0.912174 seconds.
took 0.911668 seconds.
took 0.890769 seconds.
took 0.922733 seconds.
took 0.899438 seconds.
took 0.918575 seconds.
took 0.901774 seconds.
took 0.904219 seconds.
took 1.014938 seconds.
took 1.038443 seconds.
took 1.018932 seconds.
took 1.010668 seconds.
took 1.011901 seconds.
took 1.006629 seconds.
took 1.018130 seconds.
took 1.050119 seconds.
took 1.040641 seconds.
took 1.019151 seconds.
took 1.047676 seconds.
took 1.033180 seconds.
took 1.025325 seconds.
took 1.061356 seconds.
took 1.030843 seconds.
C:/Users/BVer/are_you_ok/datavideo/p_video/kicking_k1.mp4
----- finish -----
